In [ ]:
import pystac_client
import planetary_computer as pc
import geopandas as gpd
from shapely.geometry import Point
import rasterio
from rasterio import warp, windows
from scipy import interpolate

In [ ]:
# some helper functions

def interpolation(arrays, dim0=500):
    """
    Interpolate a list of 2D arrays
    
    This is needed because the sentinel bands have different resolutions. To 
    save them into a single raster, we need to interpolate them to a shared 
    pixel x-y dimension.
    """
    result = []
    dim1 = dim0 * arrays[0].shape[1] / arrays[0].shape[0]
    
    for a in arrays:
        x = np.linspace(0, 1, a.shape[1])
        y = np.linspace(0, 1, a.shape[0])
        f = interpolate.interp2d(x, y, a, kind="linear")
        result.append(f(np.linspace(0, 1, int(dim0)),np.linspace(0, 1, int(dim1))))
    return np.stack(result)


def read_window(reader, bounds):
    """
    Read asset within bounds
    
    This allows us to read just a subset of a planetary computer asset. If the
    bounds are smaller than the full image dimension, it supports faster downloads.
    """
    bounds_ = warp.transform_bounds("epsg:4326", reader.crs, *bounds.bounds)
    window = windows.from_bounds(*bounds_, transform=reader.transform)
    return window, reader.read(window=window)


def update_profile(reader, window, count):
    """
    Define profile for writing
    
    Anytime we write a rasterio object, we have to pass in a profile. This defines
    a profile to fit in the window subregion that we care to save.
    """
    profile = reader.profile
    profile.update({
        "height": window.height,
        "width": window.width,
        "transform": windows.transform(window, reader.transform),
        "count": count
    })
    return profile


def search_catalog(bounds, date_range, constraints, collection="sentinel-2-l2a"):
    """
    Search a Collection
    
    This looks through a planetary computer collection (sentinel 2 by default) for
    scenes that match a query.
    """
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=pc.sign_inplace
    )
    return catalog.search(
        collections=collection,
        intersects=bounds, 
        datetime=date_range, 
        query=constraints
    )


def download_items(search_results, channels, out_dir=".", max_items=5):
    """
    Download Scenes
    
    This downloads from a set of planetary computer scenes. It will look for all
    assets within the channels list argument (e.g., 'B01' or 'B02' for sentinel).
    If the different assets have different dimension, they will be interpolated.
    """
    items = search_results.item_collection()
    ix = 0
    
    for item in items:
        print(f"Processing {item.id}")
        band_data = []
        for channel in channels:
            with rasterio.open(item.assets[channel].href) as reader:
                window, band = read_window(reader, bounds)
                band_data.append(band[0])
                profile = update_profile(reader, window, len(channels))
        
        band_data = interpolation(band_data)
        with rasterio.open(f"{out_dir}/{item.id}.tiff", "w", **profile) as writer:
            writer.write(np.stack(band_data))
        
        ix += 1
        if ix == max_items: break

In [ ]:
bounds = Point(86.55, 28).buffer(0.05)
date_range = "2021-01-01/2021-01-31"
constraints = {"eo:cloud_cover": {"lt": 20}, "s2:nodata_pixel_percentage": {"lt": 10}}
channels = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B11', 'B12']
search_results = search_catalog(bounds, date_range, constraints)
download_items(search_results, channels)

In [ ]:
# if want to warp after downloading
#!gdalwarp -s_srs epsg:32645 -t_srs epsg:4326 in-file.tiff out-file.tiff

In [ ]:
constraints = {}
channels = ['vv', 'vh'] # vertical and horizontal polarization
search_results = search_catalog(bounds, date_range, constraints, collection="sentinel-1-rtc")
download_items(search_results, channels)